<br/>
<img src="images/cd-logo-blue-600x600.png" alt="" width="130px" align="left"/>
<img src="images/cd-logo-blue-600x600.png" alt="" width="130px" align="right"/>
<div align="center">
<h2>Bootcamp Data Science - Módulo 3</h2><br/>
<h1>Reducción de dimensionalidad</h1>
<br/><br/>
    <b>Instructor Principal:</b> Patricio Olivares polivares@codingdojo.cl <br/>
    <b>Instructor Asistente:</b> Jesús Ortiz jortiz@codingdojo.cl<br/><br/>
    <b>Coding Dojo</b>
</div>
<br>
Fuente: "Hands-on Machine Learning with Scikit-Learn, Keras & TensorFlow"

# Reducción de dimensionalidad

- Hasta el momento, hemos visto datasets pequeños, con pocas características (columnas).
- Datasets con un mayor número de columnas son muy comunes. Estos producen:
    - Entrenamientos más lentos
    - Dificultad para encontrar una buena solución
- Es en este contexto donde se plantea la **reducción de dimensionalidad**
- Reducción de dimensionalidad viene acompañada de **pérdida de información** (ej: jpeg)


<img src="images/Webp_Jpeg_Lossless_comparative.png" alt="" width="900px"/>
<br>

Author: Christophe Mehay

# Principal Components Analysis (PCA-Análisis de Componentes Principales)

- Una de las técnicas más populares para reducción de dimensionalidad
- Identifica el hiperplano de mayor varianza respecto a los datos, y luego proyecta los datos en él.
- PCA nos permite:
    - Poder visualizar datos con alta dimensionalidad que de otra forma no sería posible
    - Reducción de dimensionalidad para mejora en la velocidad de entrenamiento
- En PCA, es importante que los datos se encuentren en la misma escala (sino, se debe **escalar**)
- PCA identifica los ejes de mayor varianza (componentes principales)

# Usando PCA para visualización

In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
import pandas as pd

cancer_data = load_breast_cancer(as_frame=True) # Para obtener datos como dataframe
print(cancer_data.target_names) # 0 Maligno, 1 benigno

In [ ]:
cancer_data.frame.head()

In [ ]:
len(cancer_data.feature_names)

In [ ]:
df = cancer_data.frame
print(df.info())
df.head()

In [ ]:
df.hist(bins=20,figsize=(15,10))
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.preprocessing import StandardScaler

X = cancer_data.data
y = cancer_data.target

scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=cancer_data.feature_names)

In [ ]:
X.head()

In [ ]:
# Todos los gráficos entre pares de columnas!
colors = {0:'red', 1:'green'}
color_map = y.map(colors)
columns = X.columns.tolist()

i = 0 
for x in columns:
    columns.remove(x)
    for y in columns:
        print("Plotting ", x, " vs ", y)
        X.plot.scatter(x=x, y=y, c=color_map)
        i += 1
print("Número de gráficos: ", i)
plt.show()


In [ ]:
from sklearn.decomposition import PCA

pca = PCA()
#pca.fit(X)
#pca.transform(X)
pcs = pca.fit_transform(X)
print(pcs.shape)
print(X.shape)

In [ ]:

plt.figure(figsize=(10,6))
plt.scatter(pcs[:,0], pcs[:,1], c = color_map)
plt.title('Gráfico 2 componentes principales')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.show()


# Usando PCA para entrenamiento

In [ ]:
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


X = cancer_data.data
y = cancer_data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify=y)

# Instanciar Escalador Estándar
scaler = StandardScaler()
# Ajustar y transformar datos
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
pca = PCA(n_components=10) # Prueba cambiando el número de componentes!!
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

In [ ]:
%%time
# Sin PCA
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

logreg.score(X_test, y_test)

In [ ]:
%%time
# Con PCA
logreg_pca = LogisticRegression()
logreg_pca.fit(X_train_pca, y_train)

logreg_pca.score(X_test_pca, y_test)

# Escogiendo cantidad de componentes PCA

Existen varios criterios para escoger los componentes más importantes
- Criterio del codo en base a la varianza (utilizar atributo ```explained_variance_ratio_``` presente en objeto PCA)

In [ ]:
pca = PCA()
X_train_pca = pca.fit_transform(X_train)

plt.figure(figsize=(10,6))
plt.plot(pca.explained_variance_ratio_,'bo-')
plt.show()

- Porcentaje de la varianza a tomar en cuenta. Este dato se entrega en el entrenamiento.
    - Ej. Si deseamos las componentes que contienen el 95% de la varianza, entrenamos el algoritmo PCA como se ve a continuación

In [ ]:
pca = PCA(n_components=0.95)
X_train_pca = pca.fit_transform(X_train)

print("El número de componentes es", pca.n_components_, "de", pca.n_features_)

# Ejemplo de uso PCA: eigenfaces

In [ ]:
from sklearn.datasets import fetch_olivetti_faces

faces = fetch_olivetti_faces()
X = faces.data
y = faces.target

fig = plt.figure(figsize=(13,13))

n = 16
for i in range(1,n+1):
    ax = fig.add_subplot(4,4,i)
    ax.imshow(X[i].reshape(64,64), cmap='gray')

plt.show()
print(X.shape)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA()
pcs = pca.fit_transform(X)

pca.components_.shape

In [ ]:
pca.singular_values_

In [ ]:
fig = plt.figure(figsize=(13,13))

n = 16
for i in range(0,n):
    ax = fig.add_subplot(4,4,i+1)
    ax.imshow(pca.components_[i].reshape(64,64), cmap='gray')

plt.show()

# Linear Discriminant Analysis (LDA)

- Es un algoritmo de clasificación, que aprende los ejes más discriminativos entre clases.
- La proyección resultante mantiene las clases tan separadas como sea posible.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
import pandas as pd

cancer_data = load_breast_cancer(as_frame=True) # Para obtener datos como dataframe
print(cancer_data.target_names) # 0 Maligno, 1 benigno

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

X = cancer_data.data
y = cancer_data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify=y)

# Instanciar Escalador Estándar
scaler = StandardScaler()
# Ajustar y transformar datos
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
lda = LinearDiscriminantAnalysis()
X_train_lda = lda.fit_transform(X_train, y_train)
lda.score(X_test, y_test)

In [ ]:
# Proyectando
colors = {0:'red', 1:'green'}
color_map = y_train.map(colors)

plt.figure(figsize=(10,6))
plt.scatter(X_train_lda,np.zeros_like(X_train_lda), c=color_map)
plt.show()

# Actividad 8

¡Hora de poner en práctica todo lo aprendido hasta ahora!

- Estudie el dataset California Housing presente en los dataset de Scikit-Learn (enlace [aquí](https://scikit-learn.org/stable/datasets/real_world.html#california-housing-dataset))
- Cree un Pipeline completo que permita predecir el precio de una casa.
- Utilice un algoritmo de reducción de dimensionalidad y utilice las nuevas componentes para predecir el precio. Compare sus resultados con su pipeline original.

In [ ]:
from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")
#codigo extra, para que imagenes de matplotlib
#estén centradas en las diapositivas, ejecutar antes de lanzar los ejemplos.